# Import All Libraries

In [7]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00


In [1]:
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os
from torch import nn
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import accelerate

sns.set()

In [3]:
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)

True


'Tesla T4'

# Ignore All Warnings

In [4]:
import warnings
warnings.filterwarnings("ignore")

# Load the DataSet

Note that the dataset has been found on Hugging Face at the link: https://huggingface.co/datasets/multi_news

In [8]:
from datasets import load_dataset

dataset = load_dataset("multi_news", split = "test")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

# Analyze and Split the Data

In [9]:
temp = dataset.to_pandas()
temp.head(3)

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...


In [10]:
data = dataset.train_test_split(test_size = 0.2)

# Load the t5 pretrained Model

This model has been extracted using the Hugging Face Library as well. Check it out on: https://huggingface.co/DunnBC22/flan-t5-base-text_summarization_data

In [11]:
tokenizer = AutoTokenizer.from_pretrained("DunnBC22/flan-t5-base-text_summarization_data")

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='DunnBC22/flan-t5-base-text_summarization_data')

In [13]:
def prepfunc(testval):

    inputs = ["Summ: " + doc for doc in testval["document"]]
    inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=testval["summary"], max_length=124, truncation=True)
    inputs["labels"] = labels["input_ids"]

    return inputs

In [14]:
tokenized_data = data.map(prepfunc, batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained("DunnBC22/flan-t5-base-text_summarization_data")

# Hyperparameter Tuning

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./res",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
)

# Training

In [29]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan
3,0.000000,nan


TrainOutput(global_step=4497, training_loss=0.0, metrics={'train_runtime': 3100.5301, 'train_samples_per_second': 4.351, 'train_steps_per_second': 1.45, 'total_flos': 1.845129508005888e+16, 'train_loss': 0.0, 'epoch': 3.0})

# Saved the Model

In [46]:
trainer.save_model("./FinalModel/")

# Loading the Model 

In [ ]:
model = trainer.from_pretrained("./FinalModel/")

# Testing the Model for a Example

In [ ]:
doc = "Climate change is an escalating global crisis with dire implications for our planet. Over the past century, Earth's average temperature has risen by approximately 1.2 degrees Celsius, predominantly due to human activities such as burning fossil fuels and deforestation. This alarming trend has led to a significant increase in extreme weather events, including hurricanes, heatwaves, and wildfires. The concentration of carbon dioxide in the atmosphere has reached around 415 parts per million, well above pre-industrial levels. The consequences are already evident, with the polar ice caps melting at an accelerated rate, resulting in rising sea levels that threaten coastal communities. Urgent action is imperative to curtail further warming and its catastrophic consequences."

summ = "Climate change is a pressing global crisis, marked by a 1.2-degree Celsius rise in Earth's average temperature over the past century due to human activities. This has led to severe weather events, increased carbon dioxide levels at 415 parts per million, and rapid polar ice cap melting causing rising sea levels. Immediate action is crucial to mitigate further devastating effects."

In [32]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=124)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  
  return predicted_summary

In [36]:
predicted_summary = predict_summary(doc)

predicted_summary

"<pad> – The unemployment rate fell sharply in March, but the numbers aren't so bad: The unemployment rate fell to 8.2%, the lowest since January 2009. The unemployment rate is expected to hold steady at 8.3%, reports MarketBeat. The unemployment rate is expected to hold steady at 8.3%, reports the Wall Street Journal. The economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate dropped to 8.2%, the lowest since January 2009. The official unemployment tally only includes those seeking work. The economy has added 858,000"